In [78]:
!pip install pytorch-lightning

In [79]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip

--2022-09-12 16:19:57--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.1’

ml-1m.zip.1         100%[===================>]   5.64M  2.95MB/s    in 1.9s    

2022-09-12 16:20:00 (2.95 MB/s) - ‘ml-1m.zip.1’ saved [5917549/5917549]

Archive:  ml-1m.zip
replace ml-1m/movies.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

https://keras.io/examples/structured_data/movielens_recommendations_transformers/

# Behavior Sequence Transformer 

In [80]:
import os
import math
import numpy as np
import pandas as pd
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from sklearn import preprocessing
import math

In [81]:
df_user = pd.read_csv("ml-1m/users.dat", sep="::", encoding="ISO-8859-1",
                      names=["user_id", "sex", "age_group", "occupation", "zip_code"],)
df_user.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,user_id,sex,age_group,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [82]:
df_rating = pd.read_csv("ml-1m/ratings.dat", sep="::", encoding="ISO-8859-1",
                        names=["user_id", "movie_id", "rating", "unix_timestamp"])
df_rating["rating"] = df_rating["rating"].astype(float)
df_rating = df_rating.sort_values(by=["unix_timestamp"]).groupby("user_id")
df_rating = pd.DataFrame(
    data = {
        "user_id": list(df_rating.groups.keys()),
        "movie_ids": list(df_rating.movie_id.apply(list)),
        "ratings": list(df_rating.rating.apply(list)),
    })

df_rating.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,user_id,movie_ids,ratings
0,1,"[3186, 1721, 1270, 1022, 2340, 1836, 3408, 120...","[4.0, 4.0, 5.0, 5.0, 3.0, 5.0, 4.0, 4.0, 5.0, ..."
1,2,"[1198, 1217, 1210, 2717, 1293, 2943, 1225, 119...","[4.0, 3.0, 4.0, 3.0, 5.0, 4.0, 5.0, 5.0, 5.0, ..."
2,3,"[593, 2858, 3534, 1968, 1961, 1431, 1266, 3671...","[3.0, 4.0, 3.0, 4.0, 4.0, 3.0, 5.0, 5.0, 4.0, ..."
3,4,"[1210, 1097, 480, 3468, 3527, 1196, 260, 1198,...","[3.0, 4.0, 4.0, 5.0, 1.0, 2.0, 5.0, 5.0, 5.0, ..."
4,5,"[2717, 919, 908, 356, 1250, 2188, 2858, 1127, ...","[1.0, 4.0, 4.0, 1.0, 5.0, 1.0, 4.0, 1.0, 2.0, ..."


In [83]:
df_movie = pd.read_csv("ml-1m/movies.dat", sep="::", encoding="ISO-8859-1",
                       names=["movie_id", "title", "genres"])

# binary class for each genres
genres = ["Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
          "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
for genre in genres:
    df_movie[genre] = df_movie["genres"].apply(lambda values: int(genre in values.split("|")))

movie_genre_map = dict(zip(df_movie['movie_id'].tolist(), df_movie[genres].values.tolist()))
df_movie.head(2)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,movie_id,title,genres,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Animation|Children's|Comedy,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [84]:
# split the movie_ids list into a set of sequences of a fixed length
sequence_length = 4
step_size = 2
def create_sequences(values, window_size, step_size):
    sequences = []
    start_index = 0
    while True:
        end_index = start_index + window_size
        seq = values[start_index:end_index]
        if len(seq) < window_size:
            seq = values[-window_size:]
            if len(seq) == window_size:
                sequences.append(seq)
            break
        sequences.append(seq)
        start_index += step_size
    return sequences

df_combine = df_rating.copy()
df_combine['movie_ids'] = df_combine['movie_ids'].apply(
    lambda ids: create_sequences(ids, sequence_length, step_size))
df_combine['ratings'] = df_combine['ratings'].apply(
    lambda ids: create_sequences(ids, sequence_length, step_size))

# explode list and combine
d1 = df_combine[["user_id", "movie_ids"]].explode("movie_ids", ignore_index=True)
d2 = df_combine[["ratings"]].explode("ratings", ignore_index=True)
df_combine = pd.concat([d1, d2], axis=1)

# join with user data
df_combine = df_combine.join(df_user.set_index("user_id"), on="user_id")
df_combine.rename(
    columns={"movie_ids": "sequence_movie_ids", "ratings": "sequence_ratings"},
    inplace=True,
)

# join with movie data
df_combine['movie_genre'] = df_combine['sequence_movie_ids'].apply(
    lambda lis: [movie_genre_map[int(x)] for x in lis])

df_combine.head()

,user_id,sequence_movie_ids,sequence_ratings,sex,age_group,occupation,zip_code,movie_genre
0,1,"[3186, 1721, 1270, 1022]","[4.0, 4.0, 5.0, 5.0]",F,1,10,48067,"[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,..."
1,1,"[1270, 1022, 2340, 1836]","[5.0, 5.0, 3.0, 5.0]",F,1,10,48067,"[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,..."
2,1,"[2340, 1836, 3408, 1207]","[3.0, 5.0, 4.0, 4.0]",F,1,10,48067,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,..."
3,1,"[3408, 1207, 2804, 260]","[4.0, 4.0, 5.0, 4.0]",F,1,10,48067,"[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,..."
4,1,"[2804, 260, 720, 1193]","[5.0, 4.0, 3.0, 5.0]",F,1,10,48067,"[[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,..."


In [85]:
# encoding
parent_map = {}
for col in ['user_id', 'sex', 'age_group', 'occupation']:
    encoder = preprocessing.LabelEncoder()
    df_combine[col]  = encoder.fit_transform(df_combine[col].tolist())
    parent_map[col] = encoder

# encoding sequence movie id
encoder = preprocessing.LabelEncoder()
encoder.fit(df_combine.explode("sequence_movie_ids")["sequence_movie_ids"].tolist())
df_combine['sequence_movie_ids'] = df_combine['sequence_movie_ids'].apply(lambda x: encoder.transform(x).tolist())
parent_map["sequence_movie_ids"] = encoder
df_combine.head()

,user_id,sequence_movie_ids,sequence_ratings,sex,age_group,occupation,zip_code,movie_genre
0,0,"[2969, 1574, 1178, 957]","[4.0, 4.0, 5.0, 5.0]",0,0,10,48067,"[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,..."
1,0,"[1178, 957, 2147, 1658]","[5.0, 5.0, 3.0, 5.0]",0,0,10,48067,"[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,..."
2,0,"[2147, 1658, 3177, 1117]","[3.0, 5.0, 4.0, 4.0]",0,0,10,48067,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,..."
3,0,"[3177, 1117, 2599, 253]","[4.0, 4.0, 5.0, 4.0]",0,0,10,48067,"[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,..."
4,0,"[2599, 253, 689, 1104]","[5.0, 4.0, 3.0, 5.0]",0,0,10,48067,"[[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,..."


In [86]:
# test train split
random_selection = np.random.rand(len(df_combine.index)) <= 0.85
train_data = df_combine[random_selection]
test_data = df_combine[~random_selection]
print(train_data.shape)
print(test_data.shape)

(423880, 8)
(74743, 8)


In [87]:
class MovieDataset(Dataset):
    """Movie dataset."""

    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        features = {
            'user_id' : row['user_id'],
            'user_sex' : row['sex'],
            'user_age' : row['age_group'],
            'user_occupation' : row['occupation'],
            'movie_history' : torch.LongTensor(row['sequence_movie_ids'][:-1]),
            'movie_rating' : torch.LongTensor(row['sequence_ratings'][:-1]),
            'movie_genre' : torch.LongTensor(row['movie_genre'][:-1]),
            'target_movie_id' : row['sequence_movie_ids'][-1],
            # 'target_rating' : row['sequence_ratings'][-1],
            'target_genre' : torch.LongTensor(row['movie_genre'][-1]),
        }

        labels = row['sequence_ratings'][-1]

        return features, labels

train_set = MovieDataset(train_data)
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_set = MovieDataset(test_data)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

### testing
sample = next(iter(train_loader))
i = 0
for key in sample[0].keys():
    print(f"{key}: {sample[0][key][i]}")

print("Label: ", sample[1][i])

user_id: 2795
user_sex: 1
user_age: 2
user_occupation: 14
movie_history: tensor([2285, 1836, 1279])
movie_rating: tensor([2, 1, 2])
movie_genre: tensor([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])
target_movie_id: 1280
target_genre: tensor([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
Label:  tensor(2., dtype=torch.float64)


In [151]:
# sequence_length

class BST(pl.LightningModule):
    def __init__(self, args=None, max_len=3):
        super().__init__()

        ### embedding layers
        self.embeddings = {}
        for feat in ['user_id', 'age_group', 'occupation', 'sex', 'sequence_movie_ids']:
            total_count = len(parent_map[feat].classes_)
            self.embeddings[feat] = torch.nn.Embedding(
                total_count + 1, 
                int(math.sqrt(total_count))
            )

        ### movie genre dense layer
        self.d_model = int(math.sqrt(len(parent_map['sequence_movie_ids'].classes_)))
        self.relu = torch.nn.ReLU()

        ### positinal embedding sequence length and output dimension
        self.pe = self.create_positional_encoding(max_len, self.d_model)
        self.linear_1 = torch.nn.Linear(78, self.d_model)

        ### 
        # self.transfomerlayer = torch.nn.TransformerEncoderLayer(72, 3, dropout=0.2)
        

    def create_positional_encoding(self, max_len, d_model, device='cpu'):
        pe = torch.zeros(max_len, d_model).to(device)
        # for each position of the word
        for pos in range(max_len):   
            # for each dimension of the each position
            for i in range(0, d_model, 2):   
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
        # include the batch size
        pe = pe.unsqueeze(0)   
        return pe
    
    def forward(
        self, user_id, user_sex, user_age, user_occupation, movie_history, 
        movie_rating, movie_genre, target_movie_id, target_genre
        ):

        ### Step 1: User features

        # embedding for all features
        user_id_emb = self.embeddings['user_id'](user_id)
        user_sex_emb = self.embeddings['sex'](user_sex)
        user_age_emb = self.embeddings['age_group'](user_age)
        user_occupation_emb = self.embeddings['occupation'](user_occupation)

        # concat all user features (m, 1, 84)
        concat_list = (user_id_emb, user_sex_emb, user_age_emb, user_occupation_emb)
        user_features = torch.cat(concat_list, 1).unsqueeze(1)

        ### Step 2: movie
        # embedding of movie and genre (m, 3, 60)
        movie_emb = self.embeddings['sequence_movie_ids'](movie_history)
        target_movie_emb = self.embeddings['sequence_movie_ids'](target_movie_id).unsqueeze(1)

        # concat movie embedding and movie genre # (m, 3, 78)
        movie_features = torch.cat((movie_emb, movie_genre), -1)   
         # (m, 3, 60)     
        movie_features =  self.linear_1(movie_features)
        movie_features = self.relu(movie_features)

        # positional embedding
        movie_features += self.pe

        # multiplication of input and rating
        # (m, 3, 60)
        multiply = movie_features * movie_rating.unsqueeze(1).permute(0, 2, 1)

        # # movie embedding
        # movie_history_emb = self.embeddings['sequence_movie_ids'](movie_history)
        # target_movie_emb = self.embeddings['sequence_movie_ids'](target_movie_id)

        return 1        

In [152]:
model = BST()
model(**sample[0])

torch.Size([64, 3, 60])


1

In [ ]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import StringLookup
from tensorflow.keras import backend as K
tf.executing_eagerly()

True

In [ ]:
# data loading and processing
users = pd.read_csv("ml-1m/users.dat", sep="::", encoding="ISO-8859-1",
                    names=["user_id", "sex", "age_group", "occupation", "zip_code"],)

ratings = pd.read_csv("ml-1m/ratings.dat", sep="::", encoding="ISO-8859-1",
                      names=["user_id", "movie_id", "rating", "unix_timestamp"])

movies = pd.read_csv("ml-1m/movies.dat", sep="::", encoding="ISO-8859-1",
                     names=["movie_id", "title", "genres"])

users["user_id"] = users["user_id"].apply(lambda x: f"user_{x}")
users["age_group"] = users["age_group"].apply(lambda x: f"group_{x}")
users["occupation"] = users["occupation"].apply(lambda x: f"occupation_{x}")

movies["movie_id"] = movies["movie_id"].apply(lambda x: f"movie_{x}")

ratings["movie_id"] = ratings["movie_id"].apply(lambda x: f"movie_{x}")
ratings["user_id"] = ratings["user_id"].apply(lambda x: f"user_{x}")
ratings["rating"] = ratings["rating"].apply(lambda x: float(x))

genres = ["Action", "Adventure", "Animation", "Children's", 
          "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
          "Film-Noir", "Horror", "Musical", "Mystery", "Romance",
          "Sci-Fi", "Thriller", "War", "Western"]

for genre in genres:
    movies[genre] = movies["genres"].apply(
        lambda values: int(genre in values.split("|")))
    
ratings_group = ratings.sort_values(by=["unix_timestamp"]).groupby("user_id")

ratings_data = pd.DataFrame(
    data={
        "user_id": list(ratings_group.groups.keys()),
        "movie_ids": list(ratings_group.movie_id.apply(list)),
        "ratings": list(ratings_group.rating.apply(list)),
        "timestamps": list(ratings_group.unix_timestamp.apply(list)),
    })
del ratings_data['timestamps']
ratings_data.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,user_id,movie_ids,ratings
0,user_1,"[movie_3186, movie_1721, movie_1270, movie_102...","[4.0, 4.0, 5.0, 5.0, 3.0, 5.0, 4.0, 4.0, 5.0, ..."
1,user_10,"[movie_597, movie_858, movie_743, movie_1210, ...","[4.0, 3.0, 3.0, 4.0, 4.0, 5.0, 5.0, 5.0, 3.0, ..."
2,user_100,"[movie_260, movie_1676, movie_1198, movie_541,...","[4.0, 3.0, 4.0, 3.0, 4.0, 3.0, 1.0, 1.0, 5.0, ..."
3,user_1000,"[movie_971, movie_260, movie_2990, movie_2973,...","[4.0, 5.0, 4.0, 3.0, 5.0, 5.0, 2.0, 5.0, 5.0, ..."
4,user_1001,"[movie_1198, movie_1617, movie_2885, movie_390...","[4.0, 4.0, 4.0, 2.0, 2.0, 1.0, 4.0, 5.0, 5.0, ..."


In [ ]:
# split the movie_ids list into a set of sequences of a fixed length
sequence_length = 4
step_size = 2
def create_sequences(values, window_size, step_size):
    sequences = []
    start_index = 0
    while True:
        end_index = start_index + window_size
        seq = values[start_index:end_index]
        if len(seq) < window_size:
            seq = values[-window_size:]
            if len(seq) == window_size:
                sequences.append(seq)
            break
        sequences.append(seq)
        start_index += step_size
    return sequences

ratings_data['movie_ids'] = ratings_data['movie_ids'].apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)

ratings_data['ratings'] = ratings_data['ratings'].apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)
ratings_data.head()

,user_id,movie_ids,ratings
0,user_1,"[[movie_3186, movie_1721, movie_1270, movie_10...","[[4.0, 4.0, 5.0, 5.0], [5.0, 5.0, 3.0, 5.0], [..."
1,user_10,"[[movie_597, movie_858, movie_743, movie_1210]...","[[4.0, 3.0, 3.0, 4.0], [3.0, 4.0, 4.0, 5.0], [..."
2,user_100,"[[movie_260, movie_1676, movie_1198, movie_541...","[[4.0, 3.0, 4.0, 3.0], [4.0, 3.0, 4.0, 3.0], [..."
3,user_1000,"[[movie_971, movie_260, movie_2990, movie_2973...","[[4.0, 5.0, 4.0, 3.0], [4.0, 3.0, 5.0, 5.0], [..."
4,user_1001,"[[movie_1198, movie_1617, movie_2885, movie_39...","[[4.0, 4.0, 4.0, 2.0], [4.0, 2.0, 2.0, 1.0], [..."


In [ ]:
# explode both movie sequence and ratings
# combine list of movie into string
ratings_data_movies = ratings_data[["user_id", "movie_ids"]].explode(
    "movie_ids", ignore_index=True
)

ratings_data_rating = ratings_data[["ratings"]].explode("ratings", ignore_index=True)
ratings_data_transformed = pd.concat([ratings_data_movies, ratings_data_rating], axis=1)

ratings_data_transformed = ratings_data_transformed.join(
    users.set_index("user_id"), on="user_id"
)
ratings_data_transformed['movie_ids'] = ratings_data_transformed['movie_ids'].apply(
    lambda x: ",".join(x)
)
ratings_data_transformed.ratings = ratings_data_transformed['ratings'].apply(
    lambda x: ",".join([str(v) for v in x])
)

del ratings_data_transformed["zip_code"]

ratings_data_transformed.rename(
    columns={"movie_ids": "sequence_movie_ids", "ratings": "sequence_ratings"},
    inplace=True,
)

print(ratings_data_transformed.shape)
ratings_data_transformed.head()

(498623, 6)


,user_id,sequence_movie_ids,sequence_ratings,sex,age_group,occupation
0,user_1,"movie_3186,movie_1721,movie_1270,movie_1022","4.0,4.0,5.0,5.0",F,group_1,occupation_10
1,user_1,"movie_1270,movie_1022,movie_2340,movie_1836","5.0,5.0,3.0,5.0",F,group_1,occupation_10
2,user_1,"movie_2340,movie_1836,movie_3408,movie_1207","3.0,5.0,4.0,4.0",F,group_1,occupation_10
3,user_1,"movie_3408,movie_1207,movie_2804,movie_260","4.0,4.0,5.0,4.0",F,group_1,occupation_10
4,user_1,"movie_2804,movie_260,movie_720,movie_1193","5.0,4.0,3.0,5.0",F,group_1,occupation_10


In [ ]:
# test train split
random_selection = np.random.rand(len(ratings_data_transformed.index)) <= 0.85
train_data = ratings_data_transformed[random_selection]
test_data = ratings_data_transformed[~random_selection]
train_data.to_csv("train_data.csv", index=False, sep="|", header=False)
test_data.to_csv("test_data.csv", index=False, sep="|", header=False)
print(train_data.shape)
print(test_data.shape)

(423564, 6)
(75059, 6)


In [ ]:
# create dataset
CSV_HEADER = list(ratings_data_transformed.columns)

CATEGORICAL_FEATURES_WITH_VOCABULARY = {
    "user_id": list(users.user_id.unique()),
    "movie_id": list(movies.movie_id.unique()),
    "sex": list(users.sex.unique()),
    "age_group": list(users.age_group.unique()),
    "occupation": list(users.occupation.unique()),
}

USER_FEATURES = ["sex", "age_group", "occupation"]
MOVIE_FEATURES = ["genres"]

def get_dataset_from_csv(csv_file_path, shuffle=False, batch_size=128):
    def process(features):
        movie_ids_string = features["sequence_movie_ids"]
        sequence_movie_ids = tf.strings.split(movie_ids_string, ",").to_tensor()

        # The last movie id in the sequence is the target movie.
        features["target_movie_id"] = sequence_movie_ids[:, -1]
        features["sequence_movie_ids"] = sequence_movie_ids[:, :-1]

        ratings_string = features["sequence_ratings"]
        sequence_ratings = tf.strings.to_number(
            tf.strings.split(ratings_string, ","), tf.dtypes.float32
        ).to_tensor()

        # The last rating in the sequence is the target for the model to predict.
        target = sequence_ratings[:, -1]
        features["sequence_ratings"] = sequence_ratings[:, :-1]

        return features, target

    dataset = tf.data.experimental.make_csv_dataset(
        csv_file_path,
        batch_size=batch_size,
        column_names=CSV_HEADER,
        num_epochs=1,
        header=False,
        field_delim="|",
        shuffle=shuffle,
    ).map(process)

    return dataset

train_dataset = get_dataset_from_csv("train_data.csv", shuffle=True, batch_size=265)
test_dataset = get_dataset_from_csv("test_data.csv", batch_size=265)

i = 10
sample = next(iter(train_dataset))
for key in sample[0].keys():
    print(f"{key}: {sample[0][key][i]}")

print('Target: ', sample[1][i])

user_id: b'user_1113'
sequence_movie_ids: [b'movie_999' b'movie_2605' b'movie_1562']
sequence_ratings: [3. 3. 1.]
sex: b'M'
age_group: b'group_18'
occupation: b'occupation_4'
target_movie_id: b'movie_2389'
Target:  tf.Tensor(3.0, shape=(), dtype=float32)


In [ ]:
def create_model_inputs():
    return {
        "user_id": layers.Input(name="user_id", shape=(1,), dtype=tf.string),
        "sequence_movie_ids": layers.Input(
            name="sequence_movie_ids", shape=(sequence_length - 1,), dtype=tf.string
        ),
        "target_movie_id": layers.Input(
            name="target_movie_id", shape=(1,), dtype=tf.string
        ),
        "sequence_ratings": layers.Input(
            name="sequence_ratings", shape=(sequence_length - 1,), dtype=tf.float32
        ),
        "sex": layers.Input(name="sex", shape=(1,), dtype=tf.string),
        "age_group": layers.Input(name="age_group", shape=(1,), dtype=tf.string),
        "occupation": layers.Input(name="occupation", shape=(1,), dtype=tf.string),
    }

# create encoder
def encode_input_features(
    inputs,
    include_user_id=True,
    include_user_features=True,
    include_movie_features=True,):

    encoded_transformer_features = []
    encoded_other_features = []

    # if True, other_feature_names = ['user_id', 'sex', 'age_group', 'occupation']
    other_feature_names = []
    if include_user_id:
        other_feature_names.append("user_id")
    if include_user_features:
        other_feature_names.extend(USER_FEATURES)

    ### Step 1: Encode user features
    for feature_name in other_feature_names:

        # Convert the string input values into integer indices.
        # (m, 1)
        vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
        idx = StringLookup(vocabulary=vocabulary, mask_token=None, num_oov_indices=0)(
            inputs[feature_name]
        )

        # Compute embedding dimensions for each feature
        # user_id 77, sex 1, age_group 2, occupation 4
        #  (m, 1) --> (m, 1, 77)
        embedding_dims = int(math.sqrt(len(vocabulary)))
        embedding_encoder = layers.Embedding(
            input_dim=len(vocabulary),
            output_dim=embedding_dims,
            name=f"{feature_name}_embedding",
        )
        encoded_other_features.append(embedding_encoder(idx))

    # Combine all user features
    # (m, 1, 84)
    if len(encoded_other_features) > 1:
        encoded_other_features = layers.concatenate(encoded_other_features)
    elif len(encoded_other_features) == 1:
        encoded_other_features = encoded_other_features[0]
    else:
        encoded_other_features = None

    ### Step 2: Create a movie embedding encoder

    # (m, 1)
    movie_vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY["movie_id"]
    movie_embedding_dims = int(math.sqrt(len(movie_vocabulary)))
    movie_index_lookup = StringLookup(
        vocabulary=movie_vocabulary,
        mask_token=None,
        num_oov_indices=0,
        name="movie_index_lookup")
    
    # Create an embedding layer with the specified dimensions
    #  (m, 1) --> (m, 1, 62)
    movie_embedding_encoder = layers.Embedding(
        input_dim=len(movie_vocabulary),
        output_dim=movie_embedding_dims,
        name=f"movie_embedding")
    
    # Create a vector lookup for movie genres.
    # (num_movies, num_genre) 
    genre_vectors = movies[genres].to_numpy()
    # (m, 1) --> (m, 1, num_genre) 
    movie_genres_lookup = layers.Embedding(
        input_dim=genre_vectors.shape[0],
        output_dim=genre_vectors.shape[1],
        embeddings_initializer=tf.keras.initializers.Constant(genre_vectors),
        trainable=False,
        name="genres_vector",
    )

    # Create a processing layer for genres.
    # --> (m, 1, 62)
    movie_embedding_processor = layers.Dense(
        units=movie_embedding_dims,
        activation="relu",
        name="process_movie_embedding_with_genres",
    )

    ## Define a function to encode a given movie id.
    def encode_movie(movie_id):
        # Convert the string input values into integer indices.
        movie_idx = movie_index_lookup(movie_id)
        movie_embedding = movie_embedding_encoder(movie_idx)
        encoded_movie = movie_embedding

        if include_movie_features:
            # (m, 1) --> (m, 1, num_genre) 
            movie_genres_vector = movie_genres_lookup(movie_idx)
            # (m, 1, 80) --> (m, 1, 62)
            encoded_movie = movie_embedding_processor(
                layers.concatenate([movie_embedding, movie_genres_vector])
            )
        return encoded_movie

    ## Encoding target_movie_id
    # (m, 1) --> (m, 1, 62)
    target_movie_id = inputs["target_movie_id"]
    encoded_target_movie = encode_movie(target_movie_id)

    ## Encoding sequence movie_ids.
    # (m, 1) --> (m, 3, 62)
    sequence_movies_ids = inputs["sequence_movie_ids"]
    encoded_sequence_movies = encode_movie(sequence_movies_ids)
    
    # Create positional embedding
    # constant (seq_length, 62)
    position_embedding_encoder = layers.Embedding(
        input_dim=sequence_length,
        output_dim=movie_embedding_dims,
        name="position_embedding",
    )
    positions = tf.range(start=0, limit=sequence_length - 1, delta=1)
    encodded_positions = position_embedding_encoder(positions)

    # Retrieve sequence ratings to incorporate them into the encoding of the movie.
    # (m, 3, 1)
    sequence_ratings = tf.expand_dims(inputs["sequence_ratings"], -1)

    # Add the positional encoding to the movie encodings and multiply them by rating.
    # (m, 3, 62)
    encoded_sequence_movies_with_poistion_and_rating = layers.Multiply()(
        [(encoded_sequence_movies + encodded_positions), sequence_ratings]
    )

    # Construct the transformer inputs
    # loop (m, 62) * 3
    for encoded_movie in tf.unstack(encoded_sequence_movies_with_poistion_and_rating, axis=1):
        encoded_transformer_features.append(tf.expand_dims(encoded_movie, 1))
    
    encoded_transformer_features.append(encoded_target_movie)

    # (m, 4, 62)
    encoded_transformer_features = layers.concatenate(
        encoded_transformer_features, axis=1
    )
    return encoded_transformer_features, encoded_other_features

# test
include_user_id = True
include_user_features = True
include_movie_features = True
hidden_units = [256, 128]
dropout_rate = 0.1
num_heads = 3

inputs = create_model_inputs()
print('Input Keys: ', inputs.keys())
transformer_features, other_features = encode_input_features(
     inputs, include_user_id, include_user_features, include_movie_features)
print("transformer_features: ", transformer_features.shape)
print("other_features: ", other_features.shape)

Input Keys:  dict_keys(['user_id', 'sequence_movie_ids', 'target_movie_id', 'sequence_ratings', 'sex', 'age_group', 'occupation'])
transformer_features:  (None, 4, 62)
other_features:  (None, 1, 84)


In [ ]:
def create_model():
    inputs = create_model_inputs()

    # (m, 4, 62), (m, 1, 84)
    transformer_features, other_features = encode_input_features(
        inputs, include_user_id, include_user_features, include_movie_features
    )

    # Create a multi-headed attention layer
    # (m, 4, 62)
    attention_output = layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=transformer_features.shape[2], dropout=dropout_rate
    )(transformer_features, transformer_features)

    # Transformer block
    # (m, 4, 62)
    attention_output = layers.Dropout(dropout_rate)(attention_output)
    # (m, 4, 62)
    x1 = layers.Add()([transformer_features, attention_output])
    x1 = layers.LayerNormalization()(x1)
    x2 = layers.LeakyReLU()(x1)

    # (m, 4, 62)
    x2 = layers.Dense(units=x2.shape[-1])(x2)
    x2 = layers.Dropout(dropout_rate)(x2)
    transformer_features = layers.Add()([x1, x2])
    transformer_features = layers.LayerNormalization()(transformer_features)

    # (m, 248)
    features = layers.Flatten()(transformer_features)

    # Included the other features.
    # (m, 248) + (m, 84) --> (m, 332)
    if other_features is not None:
        features = layers.concatenate(
            [features, layers.Reshape([other_features.shape[-1]])(other_features)]
        )

    # Fully-connected layers.
    # (m, 332) --> (m, 256) --? (m, 128)
    for num_units in hidden_units:
        features = layers.Dense(num_units)(features)
        features = layers.BatchNormalization()(features)
        features = layers.LeakyReLU()(features)
        features = layers.Dropout(dropout_rate)(features)

    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

model = create_model()

/usr/local/lib/python3.7/dist-packages/numpy/core/numeric.py:2446: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


(None, 256)
(None, 128)


In [ ]:
# training
# Compile the model.
model.compile(
    optimizer=keras.optimizers.Adagrad(learning_rate=0.01),
    loss=keras.losses.MeanSquaredError(),
    metrics=[keras.metrics.MeanAbsoluteError()],
)

# Read the training data.
train_dataset = get_dataset_from_csv("train_data.csv", shuffle=True, batch_size=265)

# Fit the model with the training data.
model.fit(train_dataset, epochs=5)

# Read the test data.
test_dataset = get_dataset_from_csv("test_data.csv", batch_size=265)

# Evaluate the model on the test data.
_, rmse = model.evaluate(test_dataset, verbose=0)
print(f"Test MAE: {round(rmse, 3)}")

Epoch 1/5
1599/1599 [==============================] - 44s 23ms/step - loss: 1.2725 - mean_absolute_error: 0.8947
Epoch 2/5
1599/1599 [==============================] - 35s 22ms/step - loss: 1.0598 - mean_absolute_error: 0.8219
Epoch 3/5
1599/1599 [==============================] - 35s 22ms/step - loss: 0.9949 - mean_absolute_error: 0.7951
Epoch 4/5
1599/1599 [==============================] - 35s 22ms/step - loss: 0.9629 - mean_absolute_error: 0.7820
Epoch 5/5
1599/1599 [==============================] - 35s 22ms/step - loss: 0.9440 - mean_absolute_error: 0.7737
Test MAE: 0.746
